In [25]:
from bs4 import BeautifulSoup
from textblob import TextBlob
import requests
import sqlite3
import datetime as dt
import pandas as pd

In [207]:
class AmazonScraper:

    def __init__(self,headers,base_url="https://www.amazon.de"):
        self.base_url = base_url
        self.headers = headers
        
    def scrape_keywords(self,keywords):
        data_list = []
        for keyword in keywords: 
            url_keyword_addition = f"/s?k={keyword}&ref=nb_sb_noss"
            r = requests.get(self.base_url+url_keyword_addition,headers=self.headers)
            soup = BeautifulSoup(r.content, "html.parser")
            groups = soup.select(".a-section.a-spacing-medium")

            for group in groups: 

                name_tag = group.select_one(".a-link-normal.a-text-normal")
                image = group.select_one(".s-image")
                stars = group.select_one(".a-icon-alt")
                price = group.select_one(".a-price-whole")

                try: 
                    price_amount = price.get_text(strip=True).replace(",",".")
                    price_amount = float(price_amount)
                except: 
                    price_amount = None

                try: 
                    source = image["src"]
                except: 
                    source = None

                try: 
                    star_amount = stars.get_text(strip=True)[:3].replace(",",".")
                    star_amount = float(star_amount)
                except: 
                    star_amount = None

                try:
                    product_name = name_tag.get_text("|",strip=True)
                except: 
                    product_name = None

                try: 
                    product_url = self.base_url + name_tag["href"]
                except: 
                    product_url = None


                data = {
                    "img source": source,
                    "stars": star_amount,
                    "price(euros)": price_amount,
                    "name": product_name,
                    "url": product_url
                }

                if any(data.values()):
                    data["keyword"] = keyword
                    data["time"] = dt.datetime.today()
                    data_list.append(data)


        return data_list

    
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36"}
scraper = AmazonScraper(headers,"https://www.amazon.de")

In [118]:
data_list = scraper.scrape_keywords(["paper","pencilcase"])
data_list

[{'img source': 'https://m.media-amazon.com/images/I/614jcodFaUL._AC_UL320_.jpg',
  'stars': 4.1,
  'price(euros)': 19.99,
  'name': 'Weedness Long-Paper Natural 50 Heftchen 32 Blatt King Size – Papers Long Longpaper Smoking Paper Rolling',
  'url': 'https://www.amazon.de/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A01395762GVQNSJP9ZNPN&url=%2FWeedness-Natural-Long-Papers-King%2Fdp%2FB079212118%2Fref%3Dsr_1_1_sspa%3Fdchild%3D1%26keywords%3Dpaper%26qid%3D1616424105%26sr%3D8-1-spons%26psc%3D1%26smid%3DA3RU9TYDAWLJON&qualifier=1616424105&id=519273705035511&widgetName=sp_atf',
  'keyword': 'paper',
  'time': datetime.datetime(2021, 3, 22, 17, 41, 48, 226887)},
 {'img source': 'https://m.media-amazon.com/images/I/71W6tTc9+NL._AC_UL320_.jpg',
  'stars': 4.4,
  'price(euros)': 10.69,
  'name': 'Tritart Origami Papier I 220 Blatt doppelseitiges Origami Faltpapier in 50 kräftigen Farben I Bastelpapier Set 80 g/m² I Je 110 Blätter 15x15cm / 10x10cm I inklusive 100 

In [94]:
len(data_list)

120

In [96]:
# creating a dataframe from our data
df = pd.DataFrame(data_list)
df.head()

,img source,stars,price(euros),name,url,keyword,time
0,https://m.media-amazon.com/images/I/71W6tTc9+N...,4.4,10.69,Tritart Origami Papier I 220 Blatt doppelseiti...,https://www.amazon.de/gp/slredirect/picassoRed...,paper,2021-03-22 15:18:14.441719
1,https://m.media-amazon.com/images/I/61tFZnuIkN...,3.9,11.95,Elements Drehmaschine konisch für Longpapers -...,https://www.amazon.de/gp/slredirect/picassoRed...,paper,2021-03-22 15:18:14.444718
2,https://m.media-amazon.com/images/I/81EGv2bACO...,4.7,11.95,100 Blatt TONPAPIER - Buntes Papier DIN A4-130...,https://www.amazon.de/gp/slredirect/picassoRed...,paper,2021-03-22 15:18:14.447716
3,https://m.media-amazon.com/images/I/71gQR0VLjJ...,4.5,11.50,50 Blatt I Vintage Kraftpapier in Grün DIN A4 ...,https://www.amazon.de/gp/slredirect/picassoRed...,paper,2021-03-22 15:18:14.452698
4,https://m.media-amazon.com/images/I/91zUyOyhUD...,4.7,6.69,OCB Premium Slim Papers 10 x 32 Blättchen,https://www.amazon.de/OCB-Premium-Slim-Papers-...,paper,2021-03-22 15:18:14.455712


In [203]:
# creating a sqlite database and writing functions to pass our DataFrame data into it
conn = sqlite3.connect("../data/products.db")

c = conn.cursor()

def drop_table():
    c.execute("drop table if exists products;")

def create_table():
    c.execute("""
    CREATE TABLE IF NOT EXISTS products 
    (
        'ProductName' VARCHAR(32) NOT NULL, 
        'Stars' FLOAT, 
        'Time' VARCHAR(255), 
        'Price(euros)' FLOAT NOT NULL, 
        'Url' VARCHAR(255), 
        'ImageSource' VARCHAR(255), 
        'Keyword' VARCHAR(255));
    """)
    
    conn.commit()
    return

def data_into_sqlite(cursor, data_list):

    for data in data_list: 
        try: 
            c.execute(f"SELECT * from products WHERE ProductName='{data['name']}'")
            item_check = c.fetchone()
        except Exception as e:
            print(e)
            continue
                
        if not item_check: 
            c.execute(f"""
            INSERT INTO products ('ProductName','Stars','Time','Price(euros)','Url','ImageSource','Keyword') 
            VALUES (
                '{data['name']}',
                '{data['stars']}',
                '{data['time']}',
                '{data['price(euros)']}',
                '{data['img source']}',
                '{data['url']}',
                '{data['keyword']}'
            );
            """)

            conn.commit()

        else: 
            c.execute(f"""DELETE FROM products WHERE ProductName='{data['name']}'""")
            c.execute(f"""
            INSERT INTO products ('ProductName','Stars','Time','Price(euros)','Url','ImageSource','Keyword') 
            VALUES (
                '{data['name']}',
                '{data['stars']}',
                '{data['time']}',
                '{data['price(euros)']}',
                '{data['img source']}',
                '{data['url']}',
                '{data['keyword']}'
            );
            """)

            conn.commit()
    return

data_into_sqlite(c,data_list)

In [176]:
# passing the data into csv files

def save_data(df,filename):
    df.to_csv(filename,index=False)

def load_data(filename): 
    df = pd.read_csv(filename)
    return df

In [152]:
# checking if the data insertion was successful
c.execute("SELECT * FROM products")
len(c.fetchall())

115

In [159]:
# now let's get some households items to pass into our scraper
r = requests.get("https://homebuyingchecklist.co/new-house-checklist-essentials/",headers=headers)
soup = BeautifulSoup(r.content, "html.parser")
text_tags = soup.select("ul li strong")

items = []
for text_tag in text_tags: 
    items.append(text_tag.get_text("-",strip=True))

items

['Set up utilities',
 'Trash bags',
 'Box cutter',
 'Basic Tools',
 'First aid kit',
 'Batteries',
 'Light bulbs',
 'Toilet paper',
 'New locks',
 'Security Safe',
 'Smart Thermostat',
 'Ethernet cables',
 'Amazon Echo',
 'Silverware',
 'Flatware sets',
 'Steak knives',
 'Silverware organizer',
 'Dishware',
 'Plates',
 'Bowls',
 'Serving platters',
 'Glasses',
 'Coffee mugs',
 'Wine glasses',
 'Utensils',
 'Tongs',
 'Spatulas',
 'Ladles and Skimmer',
 'Whisks',
 'Utensil holder',
 'Knives',
 'Cutting board',
 'Pots and Pans',
 'Baking sheets',
 'Tupperware',
 'Bottle Opener & Corkscrew',
 'Measuring Cups / spoons',
 'Television',
 'Sound bar',
 'Streaming devices for your TV –',
 'Content & Cord Cutting',
 'TV stand or Media center',
 'Couch',
 'Coffee table',
 'Coasters',
 'Other seating',
 'Arm chair',
 'Stools',
 'Poufs',
 'Rocking chair',
 'Side Table',
 'Bookshelf',
 'Throw pillows',
 'Throw blanket',
 'Area Rug']

In [169]:
# now that we have a list of items let's get a data list from it with a single function

house_items_list = scraper.scrape_keywords(items)

df = pd.DataFrame(house_items_list)

len(df)

2802

In [167]:
# first of all let's save it as a with nulls dataframe
save_data(df,"../data/with_nulls_items.csv")

# now we can get to cleanning up the data

df.isnull().sum()

img source        6
stars           279
price(euros)    372
name              0
url               0
keyword           0
time              0
dtype: int64

In [186]:
# run this code whenever you need the dataframes

nulls_df = load_data("../data/with_nulls_items.csv")
df = load_data("../data/without_nulls_items.csv")

In [187]:
# as seen in the info box above we have 372 items without price
# so we can delete them from our dataframe

df = nulls_df.dropna(how="any")
len(df)

2198

In [188]:
# so now let's save the dataframe without nulls

df

,img source,stars,price(euros),name,url,keyword,time
2,https://m.media-amazon.com/images/I/71x5kKZjbd...,3.3,0.00,GOOGLE PLAY ON KINDLE FIRE: Guide To Set Up An...,https://www.amazon.de/GOOGLE-PLAY-KINDLE-FIRE-...,Set up utilities,2021-03-22 23:04:03.529141
6,https://m.media-amazon.com/images/I/31hJEkFdRz...,4.6,248.99,Jet-line Homeoffice höhenverstellbarer Schreib...,https://www.amazon.de/Jet-Line-h%C3%B6henverst...,Set up utilities,2021-03-22 23:04:03.536134
7,https://m.media-amazon.com/images/I/51TRNzlkBg...,4.7,5.94,Emsa 513557 Stapelbare Vorratsdose für Trocken...,https://www.amazon.de/Emsa-513557-Vorratsdose-...,Set up utilities,2021-03-22 23:04:03.538133
8,https://m.media-amazon.com/images/I/51WAvuAJIg...,4.7,9.29,"Anpro Milchkännchen, Milk Pitcher 350ml / 12 f...",https://www.amazon.de/Anpro-Milchk%C3%A4nnchen...,Set up utilities,2021-03-22 23:04:03.541131
10,https://m.media-amazon.com/images/I/51JcQHDm1M...,4.8,10.99,4 x Cremetiegel 250 ml Dose aus Kunststoff wei...,https://www.amazon.de/Cremetiegel-Dose-Kunstst...,Set up utilities,2021-03-22 23:04:03.544130
...,...,...,...,...,...,...,...
2797,https://m.media-amazon.com/images/I/61tM1SMXla...,4.2,129.00,Vistefly Akku Staubsauger Kabellos Handstaubsa...,https://www.amazon.de/gp/slredirect/picassoRed...,Area Rug,2021-03-22 23:05:43.043866
2798,https://m.media-amazon.com/images/I/71Nifm3L9+...,3.6,10.99,"MojiDecor Antirutschmatte für Teppich, 24 Stüc...",https://www.amazon.de/gp/slredirect/picassoRed...,Area Rug,2021-03-22 23:05:43.047865
2799,https://m.media-amazon.com/images/I/81GNSYZjWQ...,3.4,14.99,"HETOOSHI Flächenteppiche für Wohnzimmer, flaus...",https://www.amazon.de/gp/slredirect/picassoRed...,Area Rug,2021-03-22 23:05:43.051862
2800,https://m.media-amazon.com/images/I/91UBmBridI...,3.3,13.99,HETOOSHI Plüsch Teppich Super weicher Faux Fla...,https://www.amazon.de/gp/slredirect/picassoRed...,Area Rug,2021-03-22 23:05:43.054860


In [204]:
# let's load this into a new csv

save_data(df,"../data/without_nulls_items.csv")

# now that we have 2 versions of our data in csv files 
# let's move the data into our database

conn = sqlite3.connect("../data/products.db")

c = conn.cursor()

data_into_sqlite(c,house_items_list)


near "Klassisch": syntax error
near "s": syntax error
near "s": syntax error
near "Longhi": syntax error
near "s": syntax error
near "s": syntax error
near "Blatt": syntax error
near "s": syntax error
near "s": syntax error
near "m": syntax error
near "s": syntax error
near "Guss": syntax error
near "Guss": syntax error
near "s": syntax error
near "Louis": syntax error
near "Cooper": syntax error
near "Moore": syntax error
near "s": syntax error
near "t": syntax error
near "ll": syntax error
near "art": syntax error
near "Sneaker": syntax error
near "Der": syntax error
near "Der": syntax error
near "Sand": syntax error
near "Der": syntax error
near "Gotland": syntax error
near "Artlines": syntax error
near "x5": syntax error


In [205]:
c.execute("SELECT * FROM products")
results = c.fetchall()
results

[('Weedness Long-Paper Natural 50 Heftchen 32 Blatt King Size – Papers Long Longpaper Smoking Paper Rolling',
  4.1,
  '2021-03-22 17:41:48.226887',
  19.99,
  'https://m.media-amazon.com/images/I/614jcodFaUL._AC_UL320_.jpg',
  'https://www.amazon.de/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A01395762GVQNSJP9ZNPN&url=%2FWeedness-Natural-Long-Papers-King%2Fdp%2FB079212118%2Fref%3Dsr_1_1_sspa%3Fdchild%3D1%26keywords%3Dpaper%26qid%3D1616424105%26sr%3D8-1-spons%26psc%3D1%26smid%3DA3RU9TYDAWLJON&qualifier=1616424105&id=519273705035511&widgetName=sp_atf',
  'paper'),
 ('Tritart Origami Papier I 220 Blatt doppelseitiges Origami Faltpapier in 50 kräftigen Farben I Bastelpapier Set 80 g/m² I Je 110 Blätter 15x15cm / 10x10cm I inklusive 100 Wackelaugen',
  4.4,
  '2021-03-22 17:41:48.229866',
  10.69,
  'https://m.media-amazon.com/images/I/71W6tTc9+NL._AC_UL320_.jpg',
  'https://www.amazon.de/gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&a

In [206]:
# and now we have created the complete database of amazon products with the help of the AmazonScraper class
len(results)

2713